In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('../../../data/chocolate_encoded.csv')

In [3]:
df

,Unnamed: 0,REF,Cocoa Percent,fruity,sandy,honey,spice,sour,nut,berry,...,x3_2.0,x3_2.25,x3_2.5,x3_2.6,x3_2.75,x3_3.0,x3_3.25,x3_3.5,x3_3.75,x3_4.0
0,0,5.0,0.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,15.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,15.0,0.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,15.0,0.70,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,15.0,0.82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,2357,995.0,0.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2351,2358,999.0,0.73,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2352,2359,999.0,0.72,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2353,2360,999.0,0.70,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df.dropna()

In [5]:
df['y_1.0'] = df['x3_1.0']
df['y_1.5'] = df['x3_1.5'] + df['x3_1.75']
df['y_2.0'] = df['x3_2.0'] + df['x3_2.25']
df['y_2.5'] = df['x3_2.5'] + df['x3_2.75']
df['y_3.0'] = df['x3_3.0'] + df['x3_3.25']
df['y_3.5'] = df['x3_3.5'] + df['x3_3.75']
df['y_4.0'] = df['x3_4.0']
df = df.drop(['x3_1.0','x3_1.5','x3_1.75','x3_2.0','x3_2.25','x3_2.5','x3_2.75','x3_3.0','x3_3.5','x3_3.75','x3_4.0'],axis=1)

y= df[['y_2.0','y_2.5','y_3.0','y_3.5','y_4.0']]
X = df.drop(['y_2.0','y_2.5','y_3.0','y_3.5','y_4.0'], axis =1)

<ipython-input-5-66aedaa00bf7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y_1.0'] = df['x3_1.0']
<ipython-input-5-66aedaa00bf7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y_1.5'] = df['x3_1.5'] + df['x3_1.75']
<ipython-input-5-66aedaa00bf7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [6]:
df.drop(df.loc[df['y_1.0']==1].index, inplace=True)
df.drop(df.loc[df['y_1.5']==1].index, inplace=True)

In [7]:
df

,Unnamed: 0,REF,Cocoa Percent,fruity,sandy,honey,spice,sour,nut,berry,...,x2_Vietnam,x3_2.6,x3_3.25,y_1.0,y_1.5,y_2.0,y_2.5,y_3.0,y_3.5,y_4.0
0,0,5.0,0.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,15.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,15.0,0.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,15.0,0.70,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,15.0,0.82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,2269,903.0,0.68,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2270,2270,903.0,0.68,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2271,2271,903.0,0.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2272,2272,903.0,0.70,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
df = df.drop(columns = ['y_1.0','y_1.5','Unnamed: 0'])

In [9]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, random_state=2021)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, random_state=2021)

In [10]:
df.describe()

,REF,Cocoa Percent,fruity,sandy,honey,spice,sour,nut,berry,creamy,...,x2_Vanuatu,x2_Venezuela,x2_Vietnam,x3_2.6,x3_3.25,y_2.0,y_2.5,y_3.0,y_3.5,y_4.0
count,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,...,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000,2186.000000
mean,1380.283623,0.715007,0.209973,0.152333,0.080055,0.114364,0.126715,0.166057,0.075938,0.093321,...,0.005489,0.104758,0.029735,0.000457,0.181610,0.019671,0.200366,0.392498,0.341720,0.045288
std,715.929487,0.052561,0.407382,0.359426,0.271440,0.318326,0.332730,0.372217,0.264959,0.290948,...,0.073904,0.306311,0.169893,0.021388,0.385611,0.138898,0.400366,0.488418,0.474395,0.207983
min,5.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,766.000000,0.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1450.000000,0.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1991.000000,0.740000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,2542.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
def modeling_function(model, X_train, y_train, X_val, y_val):

    # fit model on training data
    model.fit(X_train, y_train)

    # make predictions on training and validation data
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)

    # Print accuracy score
    print('Training accuracy: ', accuracy_score(y_train, train_preds))
    print('Validation accuracy: ', accuracy_score(y_val, val_preds))

    # return fitted model
    return model

In [12]:
dt = DecisionTreeClassifier(random_state=2021)

modeling_function(dt, X_train, y_train, X_val, y_val)

Training accuracy:  1.0
Validation accuracy:  0.44282238442822386


DecisionTreeClassifier(random_state=2021)

In [13]:
scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train_imputed = pd.DataFrame(imputer.transform(X_train), columns=X.columns)
X_val_imputed = pd.DataFrame(imputer.transform(X_val), columns=X.columns)

In [14]:
scaler.fit(X_train_imputed)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_imputed))
X_val_scaled = pd.DataFrame(scaler.transform(X_val_imputed))

In [15]:
dt = DecisionTreeClassifier(random_state=2021, max_depth=10)

modeling_function(dt,  X_train_scaled, y_train, X_val_scaled, y_val)

Training accuracy:  0.40227088402270883
Validation accuracy:  0.30413625304136255


DecisionTreeClassifier(max_depth=10, random_state=2021)

In [16]:
rf = RandomForestClassifier(random_state=42)

rf = modeling_function(rf, X_train, y_train, X_val, y_val)

Training accuracy:  1.0
Validation accuracy:  0.2871046228710462


In [17]:
param_grid = {'n_estimators': [21, 22, 23],
                'max_depth': [1, 3, 5, 7, 10],
                'min_samples_leaf': [1, 2, 3, 4]}

In [18]:
rf_gs = GridSearchCV(estimator=rf, param_grid=param_grid)
rf_gs.fit(X_train_scaled, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [1, 3, 5, 7, 10],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'n_estimators': [21, 22, 23]})

In [19]:
best_model = rf_gs.best_estimator_

In [20]:
rf_gs.best_params_

{'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 23}

In [21]:
modeling_function(best_model,X_train_scaled, y_train, X_val_scaled, y_val)

Training accuracy:  0.18004866180048662
Validation accuracy:  0.17518248175182483


RandomForestClassifier(max_depth=10, min_samples_leaf=2, n_estimators=23,
                       random_state=42)

In [22]:
dt2 = DecisionTreeClassifier(random_state=2021, max_depth=10)
modeling_function(dt2, X_train_scaled, y_train, X_val_scaled, y_val)

Training accuracy:  0.40227088402270883
Validation accuracy:  0.30413625304136255


DecisionTreeClassifier(max_depth=10, random_state=2021)